In [1]:
# Imports
import pandas as pd
import os
import re

In [8]:
def get_name_energy_column_name_from_number_core(number_core):
    return "CORE" + str(number_core) + "_ENERGY (J)"

def get_execution_time(data_raw):
    start_time = data_raw[TIME_COLUMN_NAME].iloc[0]
    end_time = data_raw[TIME_COLUMN_NAME].iloc[-1]
    return int(end_time - start_time)
    

TOTAL_NUMBER_CORES = 10
TOTAL_NUMBER_REPETITIONS = 30
TIME_COLUMN_NAME = "Time"

regex = re.compile('energy_metrics_(.*).m_(\d+).csv')
files_output = os.listdir("../output/")
energy_files = [k for k in files_output if 'energy_metrics_' in k]
energy_consumption_by_script = {}
execution_time_by_script = {}
for energy_file in energy_files:
    p = re.match(regex, energy_file)
    data_raw = pd.read_csv("../output/" + energy_file)
    execution_time = get_execution_time(data_raw)
    for number_core in range(TOTAL_NUMBER_CORES+1):
        start_energy = data_raw[get_name_energy_column_name_from_number_core(number_core)].iloc[0]
        end_energy = data_raw[get_name_energy_column_name_from_number_core(number_core)].iloc[-1]
        energy_consumption_core = float(end_energy - start_energy)
        #print(energy_consumption_core)
        #print("------")
        if p:
            script = p.group(1)
            if script in energy_consumption_by_script:
                energy_consumption_by_script[script] += energy_consumption_core
            else:
                energy_consumption_by_script[script] = energy_consumption_core
 
    if p:
        script = p.group(1)
        if script in execution_time_by_script:
            execution_time_by_script[script] += execution_time
        else:
            execution_time_by_script[script] = execution_time

In [9]:
print("Total Energy per script\n", energy_consumption_by_script)

Total Energy per script
 {'UCI_DataSets-monks1D_optimized_gpt4': 3284.7869720456074, 'KPMtools-optimalMatchingTest': 3288.0921630861703, 'KPMtools-optimalMatchingTest_optimized_gpt4': 3432.5236969000543, 'PSO-basic-main_optimized': 3381.9277191169676, 'private-getEnvironment_optimized': 3380.2305145270075, 'fetch_data-fetch_region_proposals_optimized': 7320.889862060547, 'IMA-IMA-main_optimized_gpt4': 4915.01690673799, 'tests-runalltests': 3872.0538330077543, 'private-getEnvironment': 3293.902496338589, 'besa-test_matlab2besa': 3417.658309937513, 'private-getEnvironment_optimized_gpt4': 3399.8738098146277, 'private-generatesessionid_optimized_gpt4': 3208.974639892578, 'adminfunc-biosigpathfirst': 3288.2227630623383, 'IMA-main': 16214.121246336785, 'PSO-basic-main_optimized_gpt4': 3486.1990051270695, 'afni-Test_m3dReorder_optimized_gpt4': 3292.2132110602106, 'emd-demo_emd': 3495.4510345453164, 'UCI_DataSets-monks1D_optimized': 3360.7108612059383, 'UCI_DataSets-monks1D': 3321.18579101510

In [10]:
print("Total Time per script\n", execution_time_by_script)

Total Time per script
 {'UCI_DataSets-monks1D_optimized_gpt4': 286900, 'KPMtools-optimalMatchingTest': 285300, 'KPMtools-optimalMatchingTest_optimized_gpt4': 313510, 'PSO-basic-main_optimized': 294706, 'private-getEnvironment_optimized': 297505, 'fetch_data-fetch_region_proposals_optimized': 865026, 'IMA-IMA-main_optimized_gpt4': 334314, 'tests-runalltests': 338514, 'private-getEnvironment': 280098, 'besa-test_matlab2besa': 286303, 'private-getEnvironment_optimized_gpt4': 290700, 'private-generatesessionid_optimized_gpt4': 268098, 'adminfunc-biosigpathfirst': 306105, 'IMA-main': 1265744, 'PSO-basic-main_optimized_gpt4': 313106, 'afni-Test_m3dReorder_optimized_gpt4': 284104, 'emd-demo_emd': 310106, 'UCI_DataSets-monks1D_optimized': 302101, 'UCI_DataSets-monks1D': 285299, 'PSO-basic-main': 306308, 'private-generatesessionid_optimized': 287503, 'afni-Test_m3dReorder_optimized': 309911, 'fetch_data-fetch_region_proposals': 1116682, 'ex6-getVocabList': 279901, 'CAM-demo': 384730, 'private-g

In [11]:
avg_energy_per_script_Dict = {}
avg_time_per_script_Dict = {}
for k,v in energy_consumption_by_script.items():
    avg_energy_per_script_Dict[k] = float(v) / float(TOTAL_NUMBER_REPETITIONS)
for k,v in avg_time_per_script_Dict.items():

In [44]:
avg_energy_per_script_Dict

{'UCI_DataSets-monks1D_optimized_gpt4': 109.49289906818692,
 'KPMtools-optimalMatchingTest': 109.60307210287235,
 'KPMtools-optimalMatchingTest_optimized_gpt4': 114.41745656333515,
 'PSO-basic-main_optimized': 112.73092397056558,
 'private-getEnvironment_optimized': 112.67435048423359,
 'fetch_data-fetch_region_proposals_optimized': 244.0296620686849,
 'IMA-IMA-main_optimized_gpt4': 163.83389689126633,
 'tests-runalltests': 129.06846110025847,
 'private-getEnvironment': 109.79674987795298,
 'besa-test_matlab2besa': 113.92194366458376,
 'private-getEnvironment_optimized_gpt4': 113.32912699382092,
 'private-generatesessionid_optimized_gpt4': 106.9658213297526,
 'adminfunc-biosigpathfirst': 109.60742543541127,
 'IMA-main': 540.4707082112261,
 'PSO-basic-main_optimized_gpt4': 116.20663350423565,
 'afni-Test_m3dReorder_optimized_gpt4': 109.74044036867369,
 'emd-demo_emd': 116.51503448484388,
 'UCI_DataSets-monks1D_optimized': 112.02369537353128,
 'UCI_DataSets-monks1D': 110.7061930338367,
 

In [3]:
data_raw

,Delta,Time,CORE0_ENERGY (J),CORE0_FREQ (MHZ),CORE0_PSTATE,CORE0_VOLT (V),CORE10_ENERGY (J),CORE10_FREQ (MHZ),CORE10_PSTATE,CORE10_VOLT (V),...,CPU_USAGE_4,CPU_USAGE_5,CPU_USAGE_6,CPU_USAGE_7,CPU_USAGE_8,CPU_USAGE_9,TOTAL_MEMORY,TOTAL_SWAP,USED_MEMORY,USED_SWAP
0,0,1717969723985,983436.919952,4700,1,0.30625,325547.350388,4700,1,0.30000,...,0.000000,0.000000,28.000000,0.000000,0.000000,4.000000,66480525312,2147479552,37298720768,0
1,199,1717969723986,983436.930649,4700,1,0.30625,325547.352097,4700,1,0.30625,...,0.000000,0.000000,28.000000,0.000000,0.000000,4.000000,66480525312,2147479552,37302333440,0
2,199,1717969724186,983437.107330,4700,1,0.30000,325547.444305,4700,1,0.30000,...,10.526316,0.000000,95.000000,0.000000,0.000000,0.000000,66480525312,2147479552,37305937920,0
3,199,1717969724386,983437.456482,4700,1,0.30000,325547.719009,4700,1,0.30000,...,10.526316,10.000000,95.238098,13.636364,5.263158,10.000000,66480525312,2147479552,37395136512,0
4,199,1717969724586,983437.591660,4700,1,0.30000,325547.747925,4700,1,0.30625,...,0.000000,0.000000,100.000000,0.000000,4.761905,5.000000,66480525312,2147479552,37406035968,0
5,199,1717969724786,983437.671219,4700,1,0.30000,325547.762375,4700,1,0.30000,...,0.000000,5.000000,100.000000,4.761905,0.000000,4.761905,66480525312,2147479552,37415186432,0
6,199,1717969724986,983437.681595,4700,1,0.30000,325547.768799,4700,1,0.30000,...,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,66480525312,2147479552,37421912064,0
7,199,1717969725186,983437.694687,4700,1,0.30000,325547.774933,4700,1,0.30000,...,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,66480525312,2147479552,37423931392,0
8,199,1717969725386,983437.707748,4700,1,0.30000,325547.792023,4700,1,0.30000,...,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,66480525312,2147479552,37427511296,0
9,199,1717969725586,983437.747040,4700,1,0.30000,325547.812683,4700,1,0.30000,...,0.000000,30.000002,100.000000,0.000000,0.000000,14.285715,66480525312,2147479552,37434753024,0
